In [1]:
import logging
import tempfile
import pandas as pd
import os
import wandb
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split

In [2]:
load_dotenv()

WANDB_API_KEY=os.environ.get('WANDB_API_KEY')
wandb.login(key=WANDB_API_KEY)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hangtn13-ssc-national-economics-university (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\HP/.netrc


True

In [3]:
# global variables

# ratio used to split train and test data
test_size = 0.2

# seed used to reproduce purposes
seed = 41

# reference (column) to stratify the data
stratify = "y"

# name of the input artifact
artifact_input_name = "hangtn13-ssc-national-economics-university/Bank-Marketing/preprocessed_data.csv:latest"

# type of the artifact
artifact_type = "segregated_data"

In [4]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

# initiate wandb project
run = wandb.init(project="Bank-Marketing", job_type="split_data")

logger.info("Downloading and reading artifact")
artifact = run.use_artifact(artifact_input_name)
artifact_dir = artifact.download()
artifact_path = os.path.join(artifact_dir, "preprocessed_data.csv")
df = pd.read_csv(artifact_path)
logger.info("Artifact downloaded")


# Split firstly in train/test, then we further divide the dataset to train and validation
logger.info("Splitting data into train and test")
splits = {}

splits["train"], splits["test"] = train_test_split(df,
                                                   test_size=test_size,
                                                   random_state=seed,
                                                   stratify=df[stratify])

# Save the artifacts. We use a temporary directory so we do not leave any trace behind
with tempfile.TemporaryDirectory() as tmp_dir:

    for split, df in splits.items():

        # Make the artifact name from the name of the split plus the provided root
        artifact_name = f"{split}.csv"

        # Get the path on disk within the temp directory
        temp_path = os.path.join(tmp_dir, artifact_name)

        logger.info(f"Uploading the {split} dataset to {artifact_name}")

        # Save then upload to W&B
        df.to_csv(temp_path,index=False)

        artifact = wandb.Artifact(name=artifact_name,
                                  type=artifact_type,
                                  description=f"{split} split of dataset {artifact_input_name}",
        )
        artifact.add_file(temp_path)

        logger.info("Logging artifact")
        run.log_artifact(artifact)

        # This waits for the artifact to be uploaded to W&B. If you
        # do not add this, the temp directory might be removed before
        # W&B had a chance to upload the datasets, and the upload
        # might fail
        artifact.wait()

10-05-2025 10:30:16 Downloading and reading artifact
10-05-2025 10:30:18 Artifact downloaded
10-05-2025 10:30:18 Splitting data into train and test
10-05-2025 10:30:18 Uploading the train dataset to train.csv
10-05-2025 10:30:19 Logging artifact
10-05-2025 10:30:35 Uploading the test dataset to test.csv
10-05-2025 10:30:35 Logging artifact


In [5]:
# close the run
# waiting a while after run the previous cell before execute this
run.finish()